# 미니프로젝트 2차 회의 준비자료

*   사용된 데이터 - https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset?select=framingham.csv
*   data preprocessing을 통해 자료의 특성을 파악/정리

[진행 과정]
1. 5개 변수 결측값 대체 (cigPerday ; 수연 /totchol, bpmeds ;민영/ glucose, bmi;은주)
2. 모델 적합해보기 (charge 변수 없이) - 결측값 처리 후에 해야할 부분

[1차 회의내용 참고사항]
- 결측치를 다 median 값으로 했음. (imputation을 그룹화해서 하면 좋을 듯.)
- 각 변수의 describe 확인, 하루 평균 담배량을 구간화하면 좋을것같다고 생각함.

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [27]:
#변수 살펴보기
data=pd.read_csv("C:/Users/SAMSUNG/Desktop/framingham.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
male               4240 non-null int64
age                4240 non-null int64
education          4135 non-null float64
currentSmoker      4240 non-null int64
cigsPerDay         4211 non-null float64
BPMeds             4187 non-null float64
prevalentStroke    4240 non-null int64
prevalentHyp       4240 non-null int64
diabetes           4240 non-null int64
totChol            4190 non-null float64
sysBP              4240 non-null float64
diaBP              4240 non-null float64
BMI                4221 non-null float64
heartRate          4239 non-null float64
glucose            3852 non-null float64
TenYearCHD         4240 non-null int64
dtypes: float64(9), int64(7)
memory usage: 530.1 KB
None


[변수설명]

Demographic:
*    성별: male or female(Nominal)
*    나이
*    교육수준

Behavioral:
*    현재 흡연여부: whether or not the patient is a current smoker (Nominal)
*    하루 평균 담배량: the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)

Information on medical history(병력):
*    BP Meds: 혈압약 복용 여부 (Nominal)
*    Prevalent Stroke: 뇌졸중 여부 (Nominal)
*    Prevalent Hyp: 고혈압 여부 (Nominal)
*    Diabetes: 당뇨병 여부 (Nominal)

Information on current medical condition:
*    Tot Chol: 총 콜레스테롤 수치 (Continuous)
*    Sys BP: systolic blood pressure (Continuous)
*    Dia BP: diastolic blood pressure (Continuous)
*    BMI: Body Mass Index (Continuous)
*    Heart Rate: 심박수 (Continuous)
*    Glucose: 포도당 수치 (Continuous)

Target variable to predict:
*    10 year risk of coronary heart disease (CHD) - (binary: “1”, means “Yes”, “0” means “No”)

In [28]:
# 결측치 빈도
data.isna().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [29]:
#겹치는 변수 - 흡연여부, 나이, 성별, bmi
#나이의 범위 - 32세~70세 (조정 필요)
data['age'].describe()

count    4240.000000
mean       49.580189
std         8.572942
min        32.000000
25%        42.000000
50%        49.000000
75%        56.000000
max        70.000000
Name: age, dtype: float64

여기까지의 결론 : education 변수는 제거하고, 결측치도 모두 제거

In [30]:
# Dropping columns education
data = data.drop(['education'], axis=1)
data = data[data['age']<=64]

In [33]:
data.describe()

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,4073.000000,4073.000000,4073.000000,4073.000000,4028.000000,4073.000000,4073.000000,4073.000000,4028.000000,4073.000000,4073.000000,4056.000000,4072.000000,3698.000000,4073.000000
mean,0.430395,48.893936,0.502823,9.299779,0.026564,0.005156,0.298797,0.024061,236.357001,131.662779,82.789344,25.762591,75.850933,81.671985,0.142892
std,0.495192,8.030427,0.500053,11.986946,0.160825,0.071628,0.457787,0.153257,44.656231,21.545966,11.830081,4.047695,11.985114,23.239373,0.350006
min,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000,83.500000,50.000000,15.960000,44.000000,40.000000,0.000000
25%,0.000000,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,205.000000,116.000000,75.000000,23.060000,68.000000,71.000000,0.000000
50%,0.000000,48.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,233.000000,128.000000,82.000000,25.375000,75.000000,78.000000,0.000000
75%,1.000000,55.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,262.000000,142.500000,89.000000,27.980000,83.000000,86.000000,0.000000
max,1.000000,64.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000


In [32]:
# 흡연자에 대한 median값으로 결측값을 채움.
data.cigsPerDay.fillna(data.cigsPerDay.where(data.currentSmoker==1).median(),inplace=True)

분류 모델 후보
    1. 로지스틱 회귀
    2. 결정트리 / 랜덤포레스트
    3. SVM
    4. KNN
    5. emsemble algorithm

https://www.kaggle.com/shikhnu/ensemble-learning-stacking-algorithm